In [1]:
# give the number of the latest FPL round
latest_gameweek = 7

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import json
import requests
import xgboost
import pickle

# allow more data columns to be shown than by default
pd.set_option('max_columns',500)

In [3]:
filepath = Path('../data/fpl/data_week' + str(latest_gameweek) + str('.csv'))
players = pd.read_csv(filepath, index_col=0)
players

,adjusted points,adjusted points per game,assists,assists_week1,assists_week2,assists_week3,assists_week4,assists_week5,assists_week6,assists_week7,bonus,bonus_week1,bonus_week2,bonus_week3,bonus_week4,bonus_week5,bonus_week6,bonus_week7,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheet_points,clean_sheets,cleansheet_week1,cleansheet_week2,cleansheet_week3,cleansheet_week4,cleansheet_week5,cleansheet_week6,cleansheet_week7,code,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,creativity,creativity_rank,creativity_rank_type,direct_freekicks_order,direct_freekicks_text,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,form 10,form 15,form 20,form 5,games played,gameweek,gameweek 10 prediction,gameweek 11 prediction,gameweek 12 prediction,gameweek 13 prediction,gameweek 14 prediction,gameweek 15 prediction,gameweek 16 prediction,gameweek 2 prediction,gameweek 3 prediction,gameweek 4 prediction,gameweek 5 prediction,gameweek 6 prediction,gameweek 7 prediction,gameweek 8 prediction,gameweek 9 prediction,goals_conceded,goals_scored,goals_week1,goals_week2,goals_week3,goals_week4,goals_week5,goals_week6,goals_week7,ict_index,ict_index_rank,ict_index_rank_type,in_dreamteam,influence,influence_rank,influence_rank_type,minutes,news,news_added,now_cost,own_goals,penalties_missed,penalties_order,penalties_saved,penalties_text,photo,points_per_game,red_cards,saves,second_name,selected_by_percent,special,squad_number,status,team,team_code,team_name,threat,threat_rank,threat_rank_type,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value,value form 10,value form 5,valuePoints metric,value_form,value_next10,value_season,web_name,xA,xA_points,xA_week1,xA_week2,xA_week3,xA_week4,xA_week5,xA_week6,xA_week7,xG,xG+xA,xG+xA per game,xGA_week1,xGA_week2,xGA_week3,xGA_week4,xGA_week5,xGA_week6,xGA_week7,xG_points,xG_week1,xG_week2,xG_week3,xG_week4,xG_week5,xG_week6,xG_week7,xPoints,xPoints week 1,xPoints week 2,xPoints week 3,xPoints week 4,xPoints week 5,xPoints week 6,xPoints week 7,xPoints_next10,yellow_cards
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,6.041078,2.013693,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,48,NaN,NaN,2.041078,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80201,NaN,NaN,0,0,-2,2,0.0,546,55,NaN,NaN,0,1,0.5,0.5,0,Bernd,0.0,1.799901,1.799901,1.799901,1.089483,3.0,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,7.9,287,20,False,79.0,172,20,270,NaN,NaN,48,0,0,NaN,0,NaN,80201.jpg,1.3,0,9,Leno,1.2,False,NaN,a,1,3,Arsenal,0.0,531,52,4,53508,1027,144304,5651,NaN,NaN,NaN,NaN,0.0,NaN,0.8,Leno,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,1.3,2.9,3.8,0.7,1.2,1.1,1.4,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,2.090127,2.220093,1.089483,NaN,NaN,NaN,NaN,NaN,0
2,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,115918,NaN,NaN,0,0,0,0,0.0,405,18,NaN,NaN,0,1,0.0,0.0,0,Rúnar Alex,0.0,NaN,NaN,NaN,NaN,NaN,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,425,37,False,0.0,419,37,0,Joined OH Leuven on a season-long loan - Expec...,2021-08-31T22:00:09.069158Z,40,0,0,NaN,0,NaN,115918.jpg,0.0,0,0,Rúnarsson,0.9,False,NaN,u,1,3,Arsenal,0.0,363,14,0,19017,0,52713,2149,NaN,NaN,NaN,NaN,0.0,NaN,0.0,Rúnarsson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47431,NaN,NaN,0,0,-2,2,0.0,606,248,NaN,NaN,0,3,0.0,0.0,0,Willian,0.0,NaN,NaN,NaN,NaN,NaN,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [4]:
teams = ['Arsenal', 'Aston Villa', 'Brentford', 'Brighton', 'Burnley', 'Chelsea', 'Crystal Palace', 'Everton',
        'Leicester City', 'Leeds United', 'Liverpool', 'Manchester City', 'Manchester Utd', 'Newcastle Utd', 
        'Norwich City', 'Southampton', 'Tottenham', 'Watford', 'West Ham', 'Wolves']

In [5]:
# get FPL fixtures data
fpl_fixtures_data = json.loads(requests.get('https://fantasy.premierleague.com/api/fixtures/').text)
fpl_fixtures = pd.DataFrame(fpl_fixtures_data)
fpl_fixtures['home team'] = [teams[i] for i in fpl_fixtures['team_h']-1]
fpl_fixtures['away team'] = [teams[i] for i in fpl_fixtures['team_a']-1]
filepath = Path('../data/fpl/fixtures.csv')
fpl_fixtures.to_csv(filepath)
fpl_fixtures

,code,event,finished,finished_provisional,id,kickoff_time,minutes,provisional_start_time,started,team_a,team_a_score,team_h,team_h_score,stats,team_h_difficulty,team_a_difficulty,pulse_id,home team,away team
0,2210271,1,True,True,1,2021-08-13T19:00:00Z,90,False,True,1,0.0,3,2.0,"[{'identifier': 'goals_scored', 'a': [], 'h': ...",3,2,66342,Brentford,Arsenal
1,2210276,1,True,True,6,2021-08-14T11:30:00Z,90,False,True,10,1.0,13,5.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",2,4,66347,Manchester Utd,Leeds United
2,2210272,1,True,True,2,2021-08-14T14:00:00Z,90,False,True,4,2.0,5,1.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",3,2,66343,Burnley,Brighton
3,2210273,1,True,True,3,2021-08-14T14:00:00Z,90,False,True,7,0.0,6,3.0,"[{'identifier': 'goals_scored', 'a': [], 'h': ...",2,5,66344,Chelsea,Crystal Palace
4,2210274,1,True,True,4,2021-08-14T14:00:00Z,90,False,True,16,1.0,8,3.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",2,4,66345,Everton,Southampton
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,2210646,38,False,False,376,2022-05-22T15:00:00Z,0,False,False,13,NaN,7,NaN,[],4,2,66717,Crystal Palace,Manchester Utd
376,2210647,38,False,False,377,2022-05-22T15:00:00Z,0,False,False,16,NaN,9,NaN,[],2,3,66718,Leicester City,Southampton
377,2210648,38,False,False,378,2022-05-22T15:00:00Z,0,False,False,20,NaN,11,NaN,[],2,5,66719,Liverpool,Wolves
378,2210649,38,False,False,379,2022-05-22T15:00:00Z,0,False,False,2,NaN,12,NaN,[],3,5,66720,Manchester City,Aston Villa


In [6]:
filepath = Path(f'../data/fbref/team_stats_week{latest_gameweek}.csv')
team_stats = pd.read_csv(filepath, index_col=0)
team_stats

,Rk,MP,W,D,L,GF,GA,GD,Pts,xG,xGA,xGD,xGD/90,Last 5,Attendance,Top Team Scorer,Goalkeeper,Notes,xG per game,xGA per game,probability no goals allowed
Squad,,,,,,,,,,,,,,,,,,,,,
Chelsea,1,7,5,1,1,15,3,12,16,11.6,8.6,3.0,0.43,D W W L W,39770,Romelu Lukaku - 3,Edouard Mendy,NaN,1.657143,1.228571,0.292710
Liverpool,2,7,4,3,0,17,6,11,15,17.9,8.1,9.9,1.41,D W W D D,52923,Mohamed Salah - 6,Alisson,NaN,2.557143,1.157143,0.314383
Manchester City,3,7,4,2,1,14,3,11,14,15.0,4.0,11.0,1.57,W W D W D,52137,"Gabriel Jesus, Ferrán Torres - 2",Ederson,NaN,2.142857,0.571429,0.564718
Manchester Utd,4,7,4,2,1,14,6,8,14,10.8,7.9,2.9,0.42,W W W L D,72879,Bruno Fernandes - 4,David de Gea,NaN,1.542857,1.128571,0.323495
Everton,5,7,4,2,1,13,8,5,14,11.3,7.3,4.0,0.57,W W L W D,35769,"Demarai Gray, Dominic Calvert-Lewin... - 3",Jordan Pickford,NaN,1.614286,1.042857,0.352446
Brighton,6,7,4,2,1,8,5,3,14,7.4,7.7,-0.3,-0.05,L W W D D,30594,Neal Maupay - 4,Robert Sánchez,NaN,1.057143,1.100000,0.332871
Brentford,7,7,3,3,1,10,6,4,12,9.6,7.5,2.2,0.31,D L W D W,16624,"Yoane Wissa, Ivan Toney... - 2",David Raya,NaN,1.371429,1.071429,0.342519
Tottenham,8,7,4,0,3,6,10,-4,12,7.3,10.1,-2.8,-0.40,W L L L W,57267,Son Heung-min - 3,Hugo Lloris,NaN,1.042857,1.442857,0.236252
West Ham,9,7,3,2,2,14,10,4,11,11.5,8.0,3.5,0.50,D D L W L,57398,Michail Antonio - 5,Łukasz Fabiański,NaN,1.642857,1.142857,0.318907


In [7]:
# upload model for making predictions
path = Path('../models/xgboost_predictor_v4_weightsAndBonus.pickle')
with open(path,'rb') as f:
    model = pickle.load(f)

In [8]:
# pre-allocate for model input
predictions = pd.DataFrame(index=players.index.copy())
input_columns = ['goalkeeper', 'defender', 'midfielder',
       'forward', 'xG90', 'xA90', 'xGA90', 'bonus', 'xPoints90',
       'games played', 'form 5', 'games_form5', 'form 10', 'games form10',
       'team xG per 90', 'team xGA per 90', 'home', 'opponent xG per 90',
       'opponent xGA per 90']
predictions[input_columns] = np.nan
predictions

,goalkeeper,defender,midfielder,forward,xG90,xA90,xGA90,bonus,xPoints90,games played,form 5,games_form5,form 10,games form10,team xG per 90,team xGA per 90,home,opponent xG per 90,opponent xGA per 90
id,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
547,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
548,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
549,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# run predictions for all players for a given time horizon
# assumes one match per gameweek at most currently?
horizon = 10
output_columns = [f'gameweek {i} prediction' for i in range(latest_gameweek+1,latest_gameweek+horizon+1)]
xGA_columns = [column for column in players.columns if 'xGA_week' in column]
xPoints_columns = [column for column in players.columns if 'xPoints week' in column]

for player_ix in players.index:
    
    # collect base info
    if players.loc[player_ix,'element_type'] == 1:
        predictions.loc[player_ix,'goalkeeper'] = 1
    elif players.loc[player_ix, 'element_type'] == 2:
        predictions.loc[player_ix,'defender'] = 1
    elif players.loc[player_ix, 'element_type'] == 3:
        predictions.loc[player_ix,'midfielder'] = 1
    elif players.loc[player_ix, 'element_type'] == 4:
        predictions.loc[player_ix,'forward'] = 1

    if players.loc[player_ix,'games played'] > 0:
        predictions.loc[player_ix, 'xG90'] = players.loc[player_ix,'xG'] / players.loc[player_ix,'games played']
        predictions.loc[player_ix, 'xA90'] = players.loc[player_ix,'xA'] / players.loc[player_ix,'games played']
        predictions.loc[player_ix, 'xGA90'] = players.loc[player_ix,xGA_columns].sum() / \
                                          players.loc[player_ix,'games played']                                        
        predictions.loc[player_ix, 'bonus'] = players.loc[player_ix, 'bps']
        predictions.loc[player_ix, 'xPoints90'] = players.loc[player_ix,'adjusted points per game']
        predictions.loc[player_ix, 'games played'] = players.loc[player_ix, 'games played']
        
        form5_points = players.loc[player_ix, xPoints_columns[-5:]].sum()
        form5_games = players.loc[player_ix, xPoints_columns[-5:]].notnull().sum()
        predictions.loc[player_ix, 'form 5'] = form5_points
        predictions.loc[player_ix, 'games_form5'] = form5_games
        
        form10_points = players.loc[player_ix, xPoints_columns[-10:]].sum()
        form10_games = players.loc[player_ix, xPoints_columns[-10:]].notnull().sum()
        predictions.loc[player_ix, 'form 10'] = form10_points
        predictions.loc[player_ix, 'games form10'] = form10_games
    else:
        predictions.loc[player_ix, 'xG90'] = 0
        predictions.loc[player_ix, 'xA90'] = 0
        predictions.loc[player_ix, 'xGA90'] = 0                       
        predictions.loc[player_ix, 'bonus'] = 0
        predictions.loc[player_ix, 'xPoints90'] = 0
        predictions.loc[player_ix, 'games played'] = 0
        predictions.loc[player_ix, 'form 5'] = 0
        predictions.loc[player_ix, 'games_form5'] = 0
        predictions.loc[player_ix, 'form 10'] = 0
        predictions.loc[player_ix, 'games form10'] = 0
        
    team = players.loc[player_ix, 'team_name']
    # team xG per 90 
    team_xG90 = team_stats.loc[team,'xG'] / team_stats.loc[team,'MP']
    predictions.loc[player_ix, 'team xG per 90'] = team_xG90
    # team xGA per 90 
    team_xGA90 = team_stats.loc[team,'xGA'] / team_stats.loc[team,'MP']
    predictions.loc[player_ix, 'team xGA per 90'] = team_xGA90
    
    for gameweek in range(latest_gameweek+1,latest_gameweek+horizon+1):

        # home games
        home_fixtures = fpl_fixtures[(fpl_fixtures['event']==gameweek) & (fpl_fixtures['home team']==team)]
        for fixture in home_fixtures.index:
            predictions.loc[player_ix, 'home'] = 1
            opponent = fpl_fixtures.loc[fixture, 'away team']
            # opponent xG per 90
            opponent_xG90 = team_stats.loc[opponent, 'xG'] / team_stats.loc[opponent,'MP']
            predictions.loc[player_ix, 'opponent xG per 90'] = opponent_xG90
            # opponent xGA per 90
            opponent_xGA90 = team_stats.loc[opponent, 'xGA'] / team_stats.loc[opponent,'MP']
            predictions.loc[player_ix, 'opponent xGA per 90'] = opponent_xGA90
            
            # predict player's points
            input_data = predictions.loc[player_ix:player_ix, input_columns]
            input_data = xgboost.DMatrix(input_data)
            prediction = model.predict(input_data)
            players.loc[player_ix, f'gameweek {gameweek} prediction'] = prediction
            
        # away game
        away_fixtures = fpl_fixtures[(fpl_fixtures['event']==gameweek) & (fpl_fixtures['away team']==team)]
        for fixture in away_fixtures.index:
            predictions.loc[player_ix, 'home'] = 0
            opponent = fpl_fixtures.loc[fixture, 'home team']
            # opponent xG per 90
            opponent_xG90 = team_stats.loc[opponent, 'xG'] / team_stats.loc[opponent,'MP']
            predictions.loc[player_ix, 'opponent xG per 90'] = opponent_xG90
            # opponent xGA per 90
            opponent_xGA90 = team_stats.loc[opponent, 'xGA'] / team_stats.loc[opponent,'MP']
            predictions.loc[player_ix, 'opponent xGA per 90'] = opponent_xGA90
            
            # predict player's points
            input_data = predictions.loc[player_ix:player_ix, input_columns]
            input_data = xgboost.DMatrix(input_data)
            prediction = model.predict(input_data)
            players.loc[player_ix, f'gameweek {gameweek} prediction'] = prediction

In [10]:
# all new predictions
players[output_columns]

,gameweek 8 prediction,gameweek 9 prediction,gameweek 10 prediction,gameweek 11 prediction,gameweek 12 prediction,gameweek 13 prediction,gameweek 14 prediction,gameweek 15 prediction,gameweek 16 prediction,gameweek 17 prediction
id,,,,,,,,,,
1,3.977383,3.965480,3.758779,4.198713,3.512175,4.151344,3.567563,3.639779,4.017078,3.573277
2,3.277598,3.265695,3.197049,3.628563,2.763469,3.681364,2.896853,2.669482,3.405957,2.803473
3,2.337055,2.217583,2.394131,2.647381,1.980933,2.694813,2.136439,2.042449,2.419406,1.982808
4,3.666691,3.685681,4.005966,4.177683,3.280962,4.398808,3.543188,3.280962,4.131710,3.280962
5,2.229051,2.119575,1.925128,2.502861,1.526286,2.274930,1.681431,1.537365,2.140664,1.537365
...,...,...,...,...,...,...,...,...,...,...
547,2.428080,2.843718,2.260516,2.485872,2.260516,3.008731,2.828578,2.103535,2.219841,2.485872
548,2.428080,2.843718,2.260516,2.485872,2.260516,3.008731,2.828578,2.103535,2.219841,2.485872
549,2.874902,3.055134,2.271759,2.998183,2.271759,3.709087,3.528933,2.045337,2.161643,2.998183


In [11]:
players['xPoints_next10'] = players[output_columns].sum(axis=1)

In [12]:
players.sort_values(f'gameweek {latest_gameweek+1} prediction', ascending=False)[['web_name','now_cost','adjusted points per game', f'gameweek {latest_gameweek+1} prediction']].head(20)

,web_name,now_cost,adjusted points per game,gameweek 8 prediction
id,,,,
233,Salah,127,8.357812,7.000152
413,Antonio,80,6.850667,6.697718
237,Alexander-Arnold,75,7.005933,6.022261
230,Mané,119,6.424526,5.985117
529,Lukaku,117,5.850000,5.626909
240,Jota,76,5.271526,5.490295
135,Chilwell,56,7.370842,5.399131
177,Calvert-Lewin,80,8.513333,5.361868
277,Fernandes,118,4.993638,5.231637


In [13]:
players.sort_values('xPoints_next10', ascending=False)[['web_name','now_cost','adjusted points per game', 'xPoints_next10']].head(20)

,web_name,now_cost,adjusted points per game,xPoints_next10
id,,,,
413,Antonio,80,6.850667,70.894950
135,Chilwell,56,7.370842,62.754489
529,Lukaku,117,5.850000,61.808534
233,Salah,127,8.357812,61.296235
177,Calvert-Lewin,80,8.513333,52.705400
237,Alexander-Arnold,75,7.005933,50.984297
205,Vardy,105,5.895857,50.385484
230,Mané,119,6.424526,49.441953
277,Fernandes,118,4.993638,49.072983


In [14]:
# save data
filepath = Path('../data/fpl/data_week' + str(latest_gameweek) + str('.csv'))
players.to_csv(filepath)